7 Лабораторная запуск в ONNX runtime

In [ ]:
import onnxruntime
#!pip install sentencepiece
import sentencepiece as spm
import numpy as np
import os
folder = r'C:\Games\lab7'
sp = spm.SentencePieceProcessor(model_file=os.path.join(folder, 'm.model'))

In [ ]:
inputs = sp.encode('привет мир , как твои дела ? что ')[-16:]
inputs = [0]*max(16 - len(inputs), 0) + inputs
inputs

Запуск сверточной нейросети

In [ ]:
sess = onnxruntime.InferenceSession(os.path.join(folder, 'Conv_next_token.onnx'))
print([(node.name, node.shape, node.type) for node in sess.get_inputs()])
token = sess.run(None, {'input.1': np.array(inputs, dtype=np.int64).reshape(1, 16)})[0]
sp.decode(inputs+[int(token[-1].argmax())])

Запуск рекуррентной

In [ ]:
sess = onnxruntime.InferenceSession(os.path.join(folder, 'LSTM_next_token.onnx'))
print([(node.name, node.shape, node.type) for node in sess.get_inputs()])
token = sess.run(None, {'input.1': np.array(inputs, dtype=np.int64).reshape(1, 16),
                        '1': np.zeros((2, 16, 256), dtype=np.float32),
                        '2': np.zeros((2, 16, 256), dtype=np.float32)})[0]
sp.decode(inputs+[int(token[-1].argmax())])

Запуск трансформера

In [ ]:
sess = onnxruntime.InferenceSession(os.path.join(folder, 'Transformer_next_token.onnx'))
print([(node.name, node.shape, node.type) for node in sess.get_inputs()])
token = sess.run(None, {'src': np.array(inputs, dtype=np.int64).reshape(1, 16)})[0]
sp.decode(inputs+[int(token[-1].argmax())])

8 лаборатораная работа. Сохранение обученной модели

In [ ]:
tokens = ['привет', 'как', 'дела', 'как', 'погода', 'азаза']
tokens = [SRC.init_token] + tokens + [SRC.eos_token]
    
src_indexes = [SRC.vocab.stoi[token] for token in tokens]

src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

src_mask = model.make_src_mask(src_tensor)

with torch.no_grad():
    enc_src = model.encoder(src_tensor, src_mask)

trg_indexes = [TRG.vocab.stoi[TRG.init_token]]

trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

trg_mask = model.make_trg_mask(trg_tensor)

with torch.no_grad():
    output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)

In [ ]:
model.eval()
inputs = (src_tensor, src_mask)
torch.onnx.export(model.encoder, inputs, 'seq2seq_enc.onnx', 
                  export_params=True,  # сохраняет веса обученных параметров внутри файла модели
                  opset_version=10,     # версия ONNX
                  do_constant_folding=True,  # следует ли выполнять укорачивание констант для оптимизации
                  input_names = ['src_tensor', 'src_mask'],   # имя входного слоя
                  output_names = ['enc_src'],  # имя выходного слоя
                  dynamic_axes={'src_tensor' : {0 : 'batch_size', 1 : 'seq_len'},    # динамичные оси
                                'src_mask' :{0: 'batch_size', 3: 'seq_len'},
                                'enc_src': {0: 'batch_size', 1: 'seq_len'}})

In [ ]:
model.eval()
inputs = (trg_tensor, enc_src, trg_mask, src_mask)
torch.onnx.export(model.decoder, inputs, 'seq2seq_dec.onnx', 
                  export_params=True,  # сохраняет веса обученных параметров внутри файла модели
                  opset_version=10,     # версия ONNX
                  do_constant_folding=True,  # следует ли выполнять укорачивание констант для оптимизации
                  input_names = ['trg_tensor', 'enc_src', 'trg_mask', 'src_mask'],   # имя входного слоя
                  output_names = ['output', 'attn'],  # имя выходного слоя
                  dynamic_axes={'trg_tensor' : {0 : 'batch_size', 1 : 'seq_len'},    # динамичные оси
                                'enc_src': {0: 'batch_size', 1: 'enc_len'},
                                'trg_mask': {0: 'batch_size', 2: 'seq_len', 3: 'seq_len'},
                                'src_mask' :{0: 'batch_size', 3: 'enc_len'},
                                'output' : {0 : 'batch_size', 1: 'seq_len'},
                                'attn': {0: 'batch_size', 2: 'seq_len'}})

In [ ]:
import pickle

# сохраняем словари для токенизации
with open('vocabs.pickle', 'wb') as f:
    pickle.dump((SRC.init_token, SRC.eos_token, dict(SRC.vocab.stoi), 
                 TRG.init_token , TRG.eos_token, dict(TRG.vocab.stoi), TRG.vocab.itos),
                f,
                protocol=pickle.HIGHEST_PROTOCOL)

Проверка модели в ONNX runtime

In [ ]:
import onnxruntime
import os
import pickle
folder = r'C:\Games\lab8'

In [ ]:
with open(os.path.join(folder, 'vocabs.pickle'), 'rb') as f:
    SRC_SOS, SRC_EOS, SRC_STOI, TRG_SOS, TRG_EOS, TRG_STOI, TRG_ITOS = pickle.load(f)

In [ ]:
sess_enc = onnxruntime.InferenceSession(os.path.join(folder, 'seq2seq_enc.onnx'))
print([(node.name, node.shape, node.type) for node in sess_enc.get_inputs()])
sess_dec = onnxruntime.InferenceSession(os.path.join(folder, 'seq2seq_dec.onnx'))
print([(node.name, node.shape, node.type) for node in sess_dec.get_inputs()])

In [ ]:
import re

def tokenize(text):
    return re.findall("[A-Z]{2,}(?![a-z])|[A-Z][a-z]+(?=[A-Z])|[\'\w\-]+",text)

def preprocess(text):
    tokens = [t.lower() for t in tokenize(text)]
    tokens = [SRC_SOS] + tokens + [SRC_EOS]
    
    src_indexes = [SRC_STOI.get(token, 0) for token in tokens]

    src_tensor = np.int64(src_indexes).reshape(1, -1)

    src_mask = (np.int64(src_indexes) != 1).reshape(1, 1, 1, -1)

    return src_tensor, src_mask

test_text = ' '.join(['женщина', 'с', 'большой', 'сумочкой', 'проходит', 'мимо', 'ворот', '.'])
src_tensor, src_mask = preprocess(test_text)

In [ ]:
enc_src = sess_enc.run(None, {'src_tensor': src_tensor,
                              'src_mask': src_mask})[0]

In [ ]:
def get_trg_mask(trg_tensor):
    trg_pad_mask = (trg_tensor != 1).reshape(1, 1, 1, -1)

    #trg_pad_mask = [batch size, 1, 1, trg len]

    trg_len = trg_tensor.shape[1]

    trg_sub_mask = np.tril(np.ones((trg_len, trg_len), dtype=np.bool))

    #trg_sub_mask = [trg len, trg len]

    return trg_pad_mask & trg_sub_mask

trg_indexes = [TRG_STOI[TRG_SOS]]

for i in range(128):
    trg_tensor = np.int64(trg_indexes).reshape(1, -1)
    
    trg_mask = get_trg_mask(trg_tensor)
    
    output, attention = sess_dec.run(None, {'trg_tensor': trg_tensor, 
                                            'enc_src': enc_src,
                                            'trg_mask': trg_mask,
                                            'src_mask': src_mask})

    pred_token = output.argmax(axis=2)[:,-1].item()

    trg_indexes.append(pred_token)

    if pred_token == TRG_STOI[TRG_EOS]:
        break

trg_tokens = [TRG_ITOS[i] for i in trg_indexes]
' '.join(trg_tokens[1:-1])